This jupyter notebook was used to develop the convolutional neural network model, which will be used to make predictions.  Fitting the model to 60,000 images, was prohibitive on my local machine, so it this was converted to 'training_app.py', which was processed in FloydHub.  The model was saved to disk to be used for future predictions. 

Handwritten Digit Recognition

Train model in Keras & Save it
Flask Backend Python Micro Framework
Deploy code to Google Cloud

#dense means fully connected layers, dropout is a technique to improve convergence, flatten to reshape our matrices for feeding into respective layers
#for convolution (images) and pooling is a technique to help choose the most relevant features in an image


In [1]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import matplotlib.pyplot as pyplot

Using TensorFlow backend.
/Users/ohm/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ohm/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ohm/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/o

In [2]:
batch_size = 128
epochs = 12
img_rows, img_cols = 28, 28
image_height = image_width = 28
square = (2, 2)
cube = (3, 3)
drpout_rt_1 = 0.2
drpout_rt_2 = 0.5
activation = 'relu'
loss='categorical_crossentropy'
optimizer=Adadelta(1.0, 0.95)
metrics=['accuracy']
activation_dense='softmax'
hl='–'*25
num_classes = 10 #y_train.shape[1]

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


-- From Constantin on stackoverflow; print variable name
```
import inspect, re

def varname(p):
  for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
    m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
    if m:
      return m.group(1)
      ```

In [4]:
print(f'The Keras data format is \t\'{K.image_data_format()}\'')
print('––––––––––––––––––––––––––––––––––––––––––––––––––––')
print(f'The shape of x_train is \t{x_train.shape}')
print(f'The shape of y_train is \t{y_train.shape}')
print(f'The shape of x_test is \t\t{x_test.shape}')
print(f'The shape of y_test is \t\t{y_test.shape}')
print('––––––––––––––––––––––––––––––––––––––––––––––––––––')

The Keras data format is 	'channels_last'
––––––––––––––––––––––––––––––––––––––––––––––––––––
The shape of x_train is 	(60000, 28, 28)
The shape of y_train is 	(60000,)
The shape of x_test is 		(10000, 28, 28)
The shape of y_test is 		(10000,)
––––––––––––––––––––––––––––––––––––––––––––––––––––


In [5]:
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    print(hl)
    print('reshaping succesful... ')
    print(f'The shape of x_train is {x_train.shape}')
    print(f'The shape of y_train is {y_train.shape}')
    print(f'The shape of x_test is {x_test.shape}')
    print(f'The shape of y_test is {y_test.shape}')
elif K.image_data_format() == 'channels_last':
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    print(hl)
    print('reshaping succesful... ')
    print(hl)
    print(f'The shape of x_train is {x_train.shape}\n\
            The shape of y_train is {y_train.shape}\n\
            The shape of x_test is {x_test.shape}\n\
            The shape of y_test is {y_test.shape}\n')
else: print('Invalid Format Submitted')

–––––––––––––––––––––––––
reshaping succesful... 
–––––––––––––––––––––––––
The shape of x_train is (60000, 28, 28, 1)
            The shape of y_train is (60000,)
            The shape of x_test is (10000, 28, 28, 1)
            The shape of y_test is (10000,)



In [6]:
#more reshaping
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [8]:
if y_train.shape[1] == y_test.shape[1]:
    print(f'There are {y_train.shape[1]} distinct categories')

There are 10 distinct categories


In [9]:
#Define Convolutional Neural Network Using keras.Sequential
def define_model():
    model = Sequential()
    model.add(Conv2D(32, 
                     kernel_size=cube, 
                     activation='relu', 
                     kernel_initializer='he_uniform', 
                     padding='same', 
                     input_shape=(image_height, image_width, 1)
                    )
             )
    model.add(MaxPooling2D(square))
    model.add(Dropout(drpout_rt_1))
    model.add(Conv2D(64, 
                     cube, 
                     activation='relu', 
                     kernel_initializer='he_uniform', 
                     padding='same'
                    )
             )
    model.add(MaxPooling2D(square))
    model.add(Dropout(drpout_rt_1))
    model.add(Conv2D(128, 
                     cube, 
                     activation='relu', 
                     kernel_initializer='he_uniform', 
                     padding='same'
                    )
             )
    model.add(MaxPooling2D(square))
    model.add(Dropout(drpout_rt_1))
    model.add(Flatten())
    model.add(Dense(128, 
                    activation='relu', 
                    kernel_initializer='he_uniform'
                   )
             )
    model.add(Dropout(drpout_rt_2))
    model.add(Dense(num_classes, activation=activation_dense))
    
    # compile model
    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics=metrics)
    return model

In [15]:


# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [10]:
model = define_model()

In [11]:
# Model is fit on 1 epoch here, but fit on 25 epochs on FloydHub.com

epochs = 1

In [12]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 1893s 32ms/step - loss: 0.4483 - accuracy: 0.8541 - val_loss: 0.0692 - val_accuracy: 0.9774


In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.06924215244501829
Test accuracy: 0.977400004863739


In [17]:
model.save('MNIST_CNN.h5')
del model
print('model saved to disk')
summarize_diagnostics(model)

model saved to disk
